# **Preparando o Ambiente**

In [1]:
!pip install pyspark #Instalar o PySpark

In [3]:
from pyspark.sql import SparkSession #Importar e criar a SparkSession

# .appName("LeituraEscrita") dá um nome para nossa aplicação
# .getOrCreate() tenta pegar uma sessão que já exista. se não houver, cria uma nova.
spark = SparkSession.builder \
.appName("LeituraEscrita_Youtube") \
.getOrCreate()

print("SparkSession iniciada!")
print(spark)

SparkSession iniciada!


# Ler os dados do arquivo "videos-stats.csv"

In [4]:
# Usei o 'spark.read.csv' para ler o arquivo
df_videos_v1 = spark.read.csv("/content/videos-stats.csv")

# Visualizar os primeiros 8 registros do arquivo

In [6]:
# A ação .show(n) exibe as 'n' primeiras linhas do DataFrame
print("Visualizando os primeiros 8 registros (leitura V1):")
df_videos_v1.show(8, truncate=False) ## truncate=False evita que o Spark corte colunas longas

Visualizando os primeiros 8 registros (leitura V1):
+----+--------------------------------------------------------------------------------------------------+-----------+------------+-------+--------+--------+---------+
|_c0 |_c1                                                                                               |_c2        |_c3         |_c4    |_c5     |_c6     |_c7      |
+----+--------------------------------------------------------------------------------------------------+-----------+------------+-------+--------+--------+---------+
|NULL|Title                                                                                             |Video ID   |Published At|Keyword|Likes   |Comments|Views    |
|0   |Apple Pay Is Killing the Physical Wallet After Only Eight Years | Tech News Briefing Podcast | WSJ|wAZZ-UWGVHI|2022-08-23  |tech   |3407.0  |672.0   |135612.0 |
|1   |The most EXPENSIVE thing I own.                                                                   |b3x28s61

# Visualizar o esquema do arquivo

In [7]:
print("Schema do DataFrame (leitura V1):")
df_videos_v1.printSchema() # O método .printSchema() mostra os nomes das colunas e seus tipos

Schema do DataFrame (leitura V1):
root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)



# Ler novamente o arquivo inferindo o esquema e visualizar o esquema novamente

In [8]:
# spark.read.option("header", "true") é o mesmo que spark.read.csv("...", header=True)
df_videos_v2 = spark.read.csv(
    "/content/videos-stats.csv", # Caminho do arquivo
    header=True, # Informa que a primeira linha é o cabeçalho
    inferSchema=True # Pede ao Spark para adivinhar os tipos de dados
)

#visualizar o novo esquema
print("Schema do DataFrame (leitura V2 - com inferSchema):")
df_videos_v2.printSchema()

#verifica como os dados ficaram
print("\nVisualizando os primeiros 8 registros (leitura V2):")
df_videos_v2.show(8)


Schema do DataFrame (leitura V2 - com inferSchema):
root
 |-- _c0: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Video ID: string (nullable = true)
 |-- Published At: date (nullable = true)
 |-- Keyword: string (nullable = true)
 |-- Likes: double (nullable = true)
 |-- Comments: double (nullable = true)
 |-- Views: double (nullable = true)


Visualizando os primeiros 8 registros (leitura V2):
+---+--------------------+-----------+------------+-------+--------+--------+---------+
|_c0|               Title|   Video ID|Published At|Keyword|   Likes|Comments|    Views|
+---+--------------------+-----------+------------+-------+--------+--------+---------+
|  0|Apple Pay Is Kill...|wAZZ-UWGVHI|  2022-08-23|   tech|  3407.0|   672.0| 135612.0|
|  1|The most EXPENSIV...|b3x28s61q3c|  2022-08-24|   tech| 76779.0|  4306.0|1758063.0|
|  2|My New House Gami...|4mgePWWCAmA|  2022-08-23|   tech| 63825.0|  3338.0|1564007.0|
|  3|Petrol Vs Liquid ...|kXiYSI7H2b0|  2022-08-23|  

# Salvar o arquivo como 'videos-parquet' no formato parquet e adicionar o cabeçalho nos dados

In [9]:
df_videos_v2.write.mode("overwrite").parquet("videos-parquet")

print("DataFrame salvo com sucesso no formato parquet na pasta 'videos-parquet'.")

DataFrame salvo com sucesso no formato parquet na pasta 'videos-parquet'.


# Ler e visualizar o arquivo 'videos-parquet' com cabeçalho nos dados

In [10]:
# Para ler Parquet, usei spark.read.parquet()
df_videos_lido_do_parquet = spark.read.parquet("videos-parquet")

#visualizar o schema e os dados
print("Schema lido do arquivo Parquet:")
df_videos_lido_do_parquet.printSchema()

print("\nDados lidos do arquivo parquet:")
df_videos_lido_do_parquet.show(8)

Schema lido do arquivo Parquet:
root
 |-- _c0: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Video ID: string (nullable = true)
 |-- Published At: date (nullable = true)
 |-- Keyword: string (nullable = true)
 |-- Likes: double (nullable = true)
 |-- Comments: double (nullable = true)
 |-- Views: double (nullable = true)


Dados lidos do arquivo parquet:
+---+--------------------+-----------+------------+-------+--------+--------+---------+
|_c0|               Title|   Video ID|Published At|Keyword|   Likes|Comments|    Views|
+---+--------------------+-----------+------------+-------+--------+--------+---------+
|  0|Apple Pay Is Kill...|wAZZ-UWGVHI|  2022-08-23|   tech|  3407.0|   672.0| 135612.0|
|  1|The most EXPENSIV...|b3x28s61q3c|  2022-08-24|   tech| 76779.0|  4306.0|1758063.0|
|  2|My New House Gami...|4mgePWWCAmA|  2022-08-23|   tech| 63825.0|  3338.0|1564007.0|
|  3|Petrol Vs Liquid ...|kXiYSI7H2b0|  2022-08-23|   tech| 71566.0|  1426.0| 922918.0|
|  4|

# Salvar o arquivo do exec. anterior como tabela chamada ‘tb_videos’ no banco de dados default do spark catalog

In [11]:
# peguei o DataFrame que acabei de ler do Parquet
#.saveAsTable("tb_videos") registra este DataFrame como uma tabela no catálogo do Spark.
df_videos_lido_do_parquet.write.mode("overwrite").saveAsTable("tb_videos")

print("DataFrame salvo como tabela 'tb_videos' no Spark Catalog.")


DataFrame salvo como tabela 'tb_videos' no Spark Catalog.


# Listar as tabelas do spark catalog para verificar a tabela

In [13]:
# spark.catalog.listTables() retorna uma lista das tabelas no catálogo
# Para exibir a lista de tabelas em formato de DataFrame, usei spark.sql("SHOW TABLES")
print("listando tabelas no catálogo:")
spark.sql("SHOW TABLES").show()

listando tabelas no catálogo:
+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|  default|tb_videos|      false|
+---------+---------+-----------+



# Utilizar o spark SQL para ler a tabela ‘tb_videos’

In [15]:
# spark.sql() permite executar qualquer comando SQL
#  fiz um SELECT simples, pegando apenas algumas colunas
df_resultado_sql = spark.sql("SELECT `Video ID`, Title, Views FROM tb_videos")

#mostrar o resultado da consulta SQL
print("Resultado da consulta SQL na 'tb_videos':")
df_resultado_sql.show(5, truncate=False)

Resultado da consulta SQL na 'tb_videos':
+-----------+--------------------------------------------------------------------------------------------------+---------+
|Video ID   |Title                                                                                             |Views    |
+-----------+--------------------------------------------------------------------------------------------------+---------+
|wAZZ-UWGVHI|Apple Pay Is Killing the Physical Wallet After Only Eight Years | Tech News Briefing Podcast | WSJ|135612.0 |
|b3x28s61q3c|The most EXPENSIVE thing I own.                                                                   |1758063.0|
|4mgePWWCAmA|My New House Gaming Setup is SICK!                                                                |1564007.0|
|kXiYSI7H2b0|Petrol Vs Liquid Nitrogen | Freezing Experiment | പെട്രോളിനെ ഐസ് ആകാൻ പറ്റുമോ | M4 Tech |         |922918.0 |
|ErMwWXQxHp0|Best Back to School Tech 2022!                                                      

# Ler o arquivo ‘comments.csv' inferindo o esquema e visualizar


In [16]:
df_comments = spark.read.csv(
    "/content/comments.csv",
    header=True,
    inferSchema=True
)

# Visualizando o schema e os dados
print("Schema do DataFrame 'comments'")
df_comments.printSchema()

print("\nVisualizando os primeiros 5 registros de 'comments':")
df_comments.show(5, truncate=False)


Schema do DataFrame 'comments'
root
 |-- _c0: string (nullable = true)
 |-- Video ID: string (nullable = true)
 |-- Comment: string (nullable = true)
 |-- Likes: string (nullable = true)
 |-- Sentiment: string (nullable = true)


Visualizando os primeiros 5 registros de 'comments':
+---+-----------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+---------+
|_c0|Video ID   |Comment                                                                                                                                                                                                       

# Salvar o arquivo como ‘comments-parquet' no formato parquet e adicionar o cabeçalho nos dados

In [17]:
# Salvando o DataFrame de comentários em formato Parquet
df_comments.write.mode("overwrite").parquet("comments-parquet")

print("DataFrame 'comments' salvo com sucesso no formato parquet na pasta 'comments-parquet'")

DataFrame 'comments' salvo com sucesso no formato parquet na pasta 'comments-parquet'
